# CMPT 732 - Fall 2021
# Keypoint detection, matching, and RANSAC

__content creator:__ Aryan Mikaeili

In [ ]:
!pip install opencv-python==4.5.4.58

In [81]:
#import modules

import cv2
import numpy as np
import matplotlib.pyplot as plt

#Practice 1: Difference of Gaussians

1. read image beaver.png
2. apply gaussian kernel using the function cv2.GaussianBlur() with 5 $\times$ 5 kernel and and two scales of 2 and 4
3. show the difference of Gaussian
4. repeat the process with different scales.

In [2]:
#read image

image = ...

#apply Gaussian kernel

image1 = ...
image2 = ...


In [3]:
#@title Solution

#read image
image = cv2.imread('images/beaver.png', cv2.IMREAD_GRAYSCALE)

#apply Gaussian kernel
image1 = cv2.GaussianBlur(image, (5, 5), 2)
image2 = cv2.GaussianBlur(image, (5,5), 4)

In [ ]:
#@title Run this cell to see results

fig, axes = plt.subplots(1, 4, figsize = (15, 15))

axes[0].imshow(image, cmap = 'gray')
axes[0].axis('off')
axes[0].set_title('Original image')

axes[1].imshow(image1, cmap = 'gray')
axes[1].axis('off')
axes[1].set_title('first filtered image')

axes[2].imshow(image2, cmap = 'gray')
axes[2].axis('off')
axes[2].set_title('second filtered image')

axes[3].imshow(image2 - image1, cmap = 'gray')
axes[3].axis('off')
axes[3].set_title('difference of gaussians')

plt.show()





#Practice 2: Sift feature detector

1. read an image (you can use the beaver that you loaded at the previous practice)
2. create a Sift feature detector using cv2.SIFT_create()
3.detect keypoints using the function detect(image, section). if your sift object is named s you should write s.detect().
4. draw the keypoints using cv2.drawKeypoints(image, keypoints, section)

In [11]:
#create sift object
sift_detector = ...

#detect keypoints
keypoints = ...

#draw keypoints

image_keys = ...


In [14]:
#@title Solution

sift_detector = cv2.SIFT_create()

keypoints = sift_detector.detect(image, None)

image_keys = cv2.drawKeypoints(image, keypoints, None)

In [ ]:
#@title Run this cell to see results

plt.imshow(image_keys, cmap = 'gray')
plt.axis('off')
plt.show()

In [ ]:
#draw rich keypoints

image_keys_rich = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.imshow(image_keys_rich, cmap = 'gray')
plt.axis('off')
plt.show()

In [ ]:
print(type(keypoints))
print(len(keypoints))

In [ ]:
kp = keypoints[0]

print(type(kp))
print(dir(kp))

print('angle', kp.angle)
print('octave', kp.octave)
print('scale', kp.size)
print('location', kp.pt)

#Practice 3: Descriptor extraction

1. on the keypoints extracted in the previous practice run compute(image, keypoints) to extract descriptors for each keypoints.
2. you can do both detection and descriptor computation at the same time using detectAndCompute()

In [ ]:
#compute descriptors

keypoints, desc = ...

In [33]:
#@title Solution

keypoints, desc = sift_detector.compute(image, keypoints)


In [ ]:
print(type(desc))
print(desc.shape)
print(desc.dtype)
print(desc[0])

#Practice 4: keypoint matching

1. read 2 images box.png, box_in_scene.png in grayscale
2. detect keypoints and compute descriptors for both images
3. instantiate a brute force matcher object using cv2.BFMatcher(matching metric) for the matching metric you can use cv2.NORM_L2
4. use the knnMatch of the bruteforce matcher object to find matches in the second image for each keypoint of the first image using the function knnMatch(descriptors1, descriptors2, k). use k = 2

5. use lowe's ration to extract good matches. set the threshold to 0.75
6. draw matches using the function cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, section). section is again None.
7. show the matches

In [ ]:
#read images
image1 = ...
image2 = ...

#detect keypoints and extract descriptors
kp1, desc1 = ...
kp2, desc2 = ...

#matcher class
bf_matcher = ...

#knn matching 
matches = ...

#finding good matches
good_matches = []
...

print('%d good matches'%len(good_matches))

#draw matches
image_matches = ...

In [ ]:
#@title Solution

#read images
image1 = cv2.imread('images/box.png', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('images/box_in_scene.png', cv2.IMREAD_GRAYSCALE)

#detect keypoints and extract descriptors
kp1, desc1 = sift_detector.detectAndCompute(image1, None)
kp2, desc2 = sift_detector.detectAndCompute(image2, None)

#matcher class
bf_matcher = cv2.BFMatcher(cv2.NORM_L2)

#knn matching 
matches = bf_matcher.knnMatch(desc1, desc2, 2)

#finding good matches
good_matches = []

for m in matches:
  if m[0].distance / m[1].distance < 0.75:
    good_matches.append(m[0])

print('%d good matches'%len(good_matches))

#draw matches
image_matches = cv2.drawMatches(image1, kp1, image2, kp2, good_matches, None)

In [ ]:
#@title Run this cell to see results
fig, axes = plt.subplots(1, 3, figsize = (30, 30))

axes[0].imshow(image1, cmap = 'gray')
axes[0].set_title('Image 1')
axes[0].axis('off')

axes[1].imshow(image2, cmap = 'gray')
axes[1].set_title('Image 2')
axes[1].axis('off')

axes[2].imshow(image_matches, cmap = 'gray')
axes[2].set_title('matches')
axes[2].axis('off')

plt.show()

#Practice 5: matching using Hellinger distance

1. read 2 images box.png, box_in_scene.png in grayscale
2. detect keypoints and compute descriptors for both images
3. Normalize the descriptors so that $|d_i| = 1$.
4. compute the Hellinger distance matrix $A_{nm}$ in which $a_{ij} = \sqrt{1 - \sqrt{d_i.d_j}}$ 
5. use lowe's ratio to find good matches. you can make match objects using cv2.DMatch(queryIdx, trainIdx, imgIdx, distance)
6. draw matches using the function cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, section). section is again None.
7. show the matches

In [ ]:
#Normalize descriptors
desc1_N= ...
desc2_N = ...

#compute the distance matrix
D_hell = ...

#finding good matches
good_matches = []
...

print('%d good matches'%len(good_matches))

#draw matches
image_matches = ...



In [141]:
#@title Solution

desc1_N = desc1 / np.linalg.norm(desc1, axis = 1, keepdims=True)
desc2_N = desc2 / np.linalg.norm(desc2, axis = 1, keepdims=True)

D_hell = np.sqrt(1 - np.sqrt(desc1_N @ desc2_N.T))

good_matches = []
for i in range(desc1_N.shape[0]):
  sorted_idx = np.argsort(D_hell[i])
  d1 = D_hell[i][sorted_idx[0]]
  d2 = D_hell[i][sorted_idx[1]]

  if d1 / d2 < 0.75:
    match = cv2.DMatch(i, sorted_idx[0],0, d1)
    good_matches.append(match)
  

image_matches = cv2.drawMatches(image1, kp1, image2, kp2, good_matches, None)


In [ ]:
#@title Run this cell to see results
fig, axes = plt.subplots(1, 3, figsize = (30, 30))

axes[0].imshow(image1, cmap = 'gray')
axes[0].set_title('Image 1')
axes[0].axis('off')

axes[1].imshow(image2, cmap = 'gray')
axes[1].set_title('Image 2')
axes[1].axis('off')

axes[2].imshow(image_matches, cmap = 'gray')
axes[2].set_title('matches')
axes[2].axis('off')

plt.show()